In [1]:
import pandas as pd
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
import geopy.distance
import numpy as np

In [2]:
train_file = 'data/in/train.csv'
test_file = 'data/in/test.csv'

train_data = pd.read_csv(train_file,index_col=0)
test_data = pd.read_csv(test_file,index_col=0)

In [3]:
train_data.head()

,FECHA,LATITUD_ORIGEN,LONGITUD_ORIGEN,LATITUD_DESTINO,LONGITUD_DESTINO,DISTANCIA,TIEMPO
ID,,,,,,,
124092420,2021-07-28,-11.965070,-77.12269,-12.07020,-77.13524,14747.0,1775.0
124091219,2021-07-25,-12.155090,-76.97996,-12.07570,-76.91754,17171.0,1821.0
124091187,2021-07-25,-14.533770,-75.18460,-14.88630,-74.99414,58708.0,3842.0
124093495,2021-08-01,-5.201517,-80.64635,-5.38453,-80.62314,23611.0,1446.0
124092156,2021-07-27,-11.967470,-77.12508,-12.03500,-77.07444,14229.0,1974.0


In [4]:
la_orig = train_data['LATITUD_ORIGEN'].values
lo_orig = train_data['LONGITUD_ORIGEN'].values
la_dest = train_data['LATITUD_DESTINO'].values
lo_dest = train_data['LONGITUD_DESTINO'].values

distances = []
for la1, lo1, la2, lo2 in zip(la_orig,lo_orig,la_dest,lo_dest):
    dist = geopy.distance.distance((la1,lo1),(la2,lo2)).m
    distances.append(dist)

train_data['EARTH_DISTANCE'] = distances

In [5]:
train_data.head()

,FECHA,LATITUD_ORIGEN,LONGITUD_ORIGEN,LATITUD_DESTINO,LONGITUD_DESTINO,DISTANCIA,TIEMPO,EARTH_DISTANCE
ID,,,,,,,,
124092420,2021-07-28,-11.965070,-77.12269,-12.07020,-77.13524,14747.0,1775.0,11709.759288
124091219,2021-07-25,-12.155090,-76.97996,-12.07570,-76.91754,17171.0,1821.0,11104.026667
124091187,2021-07-25,-14.533770,-75.18460,-14.88630,-74.99414,58708.0,3842.0,44070.222890
124093495,2021-08-01,-5.201517,-80.64635,-5.38453,-80.62314,23611.0,1446.0,20401.135922
124092156,2021-07-27,-11.967470,-77.12508,-12.03500,-77.07444,14229.0,1974.0,9285.408508


In [6]:
la_orig = test_data['LATITUD_ORIGEN'].values
lo_orig = test_data['LONGITUD_ORIGEN'].values
la_dest = test_data['LATITUD_DESTINO'].values
lo_dest = test_data['LONGITUD_DESTINO'].values

distances = []
for la1, lo1, la2, lo2 in zip(la_orig,lo_orig,la_dest,lo_dest):
    dist = geopy.distance.distance((la1,lo1),(la2,lo2)).m
    distances.append(dist)

test_data['EARTH_DISTANCE'] = distances

In [7]:
test_data.head()

,FECHA,LATITUD_ORIGEN,LONGITUD_ORIGEN,LATITUD_DESTINO,LONGITUD_DESTINO,EARTH_DISTANCE
ID,,,,,,
124089165,2021-07-20,-12.781050,-74.99228,-12.76703,-74.64612,37618.751360
124093149,2021-07-31,-7.140513,-78.51395,-7.58284,-78.07524,68841.223973
124090750,2021-07-24,-12.010250,-77.11206,-11.99380,-77.02614,9532.156469
124092491,2021-07-29,-12.155750,-76.97977,-12.22090,-76.91364,10185.051111
124091871,2021-07-27,-12.114920,-77.04678,-12.07910,-77.11724,8633.569104


In [8]:
X=train_data.iloc[:,[False,True,True,True,True,False,False,True]].values
y=train_data.iloc[:,5:-1].values
yhat=test_data.iloc[:,1:].values

In [9]:
from sklearn import preprocessing
preprocessing.minmax_scale(X)
preprocessing.minmax_scale(yhat)

array([[0.36037929, 0.52007466, 0.36614227, 0.54753148, 0.03653944],
       [0.74842017, 0.22806174, 0.72358461, 0.26564229, 0.06726531],
       [0.4134065 , 0.34430488, 0.41945535, 0.35188306, 0.00889957],
       ...,
       [0.40340095, 0.3552709 , 0.41559216, 0.34470825, 0.01789533],
       [0.40340026, 0.3552709 , 0.42098601, 0.39107818, 0.04980804],
       [0.15010318, 0.93230005, 0.23882396, 0.90311374, 0.13651709]])

In [10]:
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
    model.compile(loss='mape', optimizer='adam')
    return model

In [11]:
n_inputs, n_outputs = X.shape[1], y.shape[1]
print(X.shape[1], y.shape[1])

5 2


In [12]:
model = get_model(n_inputs, n_outputs)

In [13]:
history = model.fit(X, y, verbose=0, epochs=200)

In [14]:
ypred = model.predict(yhat)

In [15]:
print(ypred)

[[ 49808.57     3749.255 ]
 [ 90937.14     6017.9556]
 [ 12792.947    1563.3795]
 ...
 [ 24849.611    2367.875 ]
 [ 67576.234    4755.1216]
 [183663.17    11222.843 ]]


In [16]:
model.evaluate(X,y)

63/63 [==============================] - 0s 452us/step - loss: 18.7464


18.746431350708008

In [17]:
ids = pd.read_csv(test_file)
ids_1 = ids['ID']

In [18]:
df = pd.DataFrame()
df['ID'] = ids_1
df['DISTANCIA'] = ypred[:,0]
df['TIEMPO'] = ypred[:,1]
print(df)

            ID      DISTANCIA        TIEMPO
0    124089165   49808.570312   3749.254883
1    124093149   90937.140625   6017.955566
2    124090750   12792.947266   1563.379517
3    124092491   13655.318359   1631.398071
4    124091871   11606.814453   1471.943481
..         ...            ...           ...
847  124088805   13309.388672   1603.223999
848  124089757   11564.927734   1468.220459
849  124089569   24849.611328   2367.875000
850  124092518   67576.234375   4755.121582
851  124090644  183663.171875  11222.842773

[852 rows x 3 columns]


In [19]:
df.to_csv('data/out/try2.csv',index=False)

In [20]:
model.save('data/out/try2.h5')